In [4]:
import docs
from minsearch import Index

from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
github_data = docs.read_github_data()
pased_data = docs.parse_data(github_data)
doc_chunks =docs.chunk_documents(pased_data)

In [6]:
dimensionality = 512
model_name = "jinaai/jina-embeddings-v2-small-en"
collection_name = "evidently-docs"


# if you want to re-insert the data, delete the collection first
# qd_client.delete_collection(collection_name=collection_name)


qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=dimensionality,
        distance=models.Distance.COSINE
    )
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `evidently-docs` already exists!"},"time":0.000521965}'

In [7]:
points = []

for i, doc in enumerate(doc_chunks):
    text = doc.get('title', '') + ' ' + doc.get('description', '') + ' ' + doc['content']
    text = text.strip()
    vector = models.Document(text=text, model=model_name)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)


qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [8]:
def vector_search(query, num_results=5):    
    vector = models.Document(text=query, model=model_name)

    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=vector,
        limit=num_results,
        with_payload=True
    )

    results = []

    for point in query_points.points:
        results.append(point.payload)

    return results

In [9]:
def search(query):
    results = vector_search(
        query=query,
        num_results=15
    )

    return results

In [11]:
query = "How do I use LLM as a judge?"
search_results = search(query)

In [12]:
search_results

[{'start': 0,
  'content': 'import CloudSignup from \'/snippets/cloud_signup.mdx\';\nimport CreateProject from \'/snippets/create_project.mdx\';\n\nIn this tutorial, we\'ll show how to evaluate text for custom criteria using LLM as the judge, and evaluate the LLM judge itself.\n\n<Info>\n  **This is a local example.** You will run and explore results using the open-source Python library. At the end, we’ll optionally show how to upload results to the Evidently Platform for easy exploration.\n</Info>\n\nWe\'ll explore two ways to use an LLM as a judge:\n\n- **Reference-based**. Compare new responses against a reference. This is useful for regression testing or whenever you have a "ground truth" (approved responses) to compare against.\n- **Open-ended**. Evaluate responses based on custom criteria, which helps evaluate new outputs when there\'s no reference available.\n\nWe will focus on demonstrating **how to create and tune the LLM evaluator**, which you can then apply in different cont